# Chapter 25: CI/CD and Project Organization

A professional Python project needs more than just code. This notebook covers the
infrastructure around your code: pre-commit hooks, GitHub Actions CI/CD, project
structure, Makefiles, dependency management, and release workflows.

## Topics Covered
- **Pre-commit hooks**: Configuration and common hooks
- **GitHub Actions**: Workflow YAML structure, jobs, steps
- **CI pipeline**: Lint, type-check, test (parallel jobs)
- **Branch protection** and required status checks
- **Project structure**: src-layout, tests, docs, Makefile
- **Makefile patterns**: Prereqs, install, check, test
- **Dependency management**: Lock files, security scanning
- **Release workflow**: Tagging, changelog, semantic versioning

## Pre-commit Hooks

**Pre-commit** is a framework for managing git hook scripts that run automatically
before each commit. It catches issues early, before code enters the repository.

How it works:
1. Install `pre-commit`: `pip install pre-commit`
2. Create a `.pre-commit-config.yaml` configuration file
3. Install the hooks: `pre-commit install`
4. Hooks run automatically on `git commit`

In [ ]:
# .pre-commit-config.yaml: Typical configuration

pre_commit_config = """
# .pre-commit-config.yaml
repos:
  # General file checks
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v5.0.0
    hooks:
      - id: trailing-whitespace        # Remove trailing whitespace
      - id: end-of-file-fixer          # Ensure files end with newline
      - id: check-yaml                 # Validate YAML syntax
      - id: check-toml                 # Validate TOML syntax
      - id: check-added-large-files    # Prevent committing large files
        args: ['--maxkb=500']
      - id: check-merge-conflict       # Detect unresolved merge markers
      - id: debug-statements           # Flag print()/pdb left in code

  # Ruff: linting and formatting
  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.8.6
    hooks:
      - id: ruff                       # Lint
        args: ['--fix']                # Auto-fix safe issues
      - id: ruff-format                # Format

  # Type checking with mypy
  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.14.1
    hooks:
      - id: mypy
        additional_dependencies: []    # Add stubs here if needed
"""

print("Typical .pre-commit-config.yaml:")
print(pre_commit_config)

print("Common pre-commit commands:")
print("  pre-commit install            # Install hooks into .git/hooks")
print("  pre-commit run --all-files    # Run all hooks on all files")
print("  pre-commit autoupdate         # Update hook versions")
print("  git commit --no-verify        # Skip hooks (emergency only)")

In [ ]:
# Pre-commit hook lifecycle: what happens on git commit

hook_lifecycle = """
Pre-commit hook lifecycle:

  $ git add my_file.py
  $ git commit -m "Add feature"
  
  [pre-commit] running hooks...
  
  Step 1: trailing-whitespace ..... Passed
  Step 2: end-of-file-fixer ...... Passed
  Step 3: check-yaml ............. Passed  (skipped - no YAML files staged)
  Step 4: ruff ................... Failed
           - src/main.py:10:1 F401 'os' imported but unused
           - Fixed 1 error (auto-fix applied)
  Step 5: ruff-format ........... Failed
           - 1 file reformatted
  Step 6: mypy .................. Passed
  
  [pre-commit] Some hooks modified files.
  [pre-commit] Commit aborted. Review changes and re-commit.
  
  $ git diff                     # See what hooks changed
  $ git add my_file.py           # Stage the fixes
  $ git commit -m "Add feature"  # Retry -- all hooks pass now
  
  [pre-commit] running hooks...
  All hooks passed.
  [main abc1234] Add feature
"""

print(hook_lifecycle)

## GitHub Actions: CI/CD Workflows

**GitHub Actions** is a CI/CD platform built into GitHub. Workflows are defined in
YAML files under `.github/workflows/`. Each workflow consists of **jobs** that contain
**steps**, and jobs can run in parallel or depend on each other.

Key concepts:
- **Workflow**: Triggered by events (push, PR, schedule, etc.)
- **Job**: Runs on a virtual machine (runner)
- **Step**: Individual command or action within a job
- **Action**: Reusable unit of CI logic (e.g., `actions/checkout@v4`)

In [ ]:
# GitHub Actions workflow: CI pipeline with parallel jobs

ci_workflow = """
# .github/workflows/ci.yml
name: CI

on:
  push:
    branches: [main]
  pull_request:
    branches: [main]

# Cancel in-progress runs for the same branch
concurrency:
  group: ci-${{ github.ref }}
  cancel-in-progress: true

jobs:
  # --- Job 1: Lint ---
  lint:
    name: Lint
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.12"

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install ruff

      - name: Run ruff check
        run: ruff check .

      - name: Run ruff format check
        run: ruff format --check .

  # --- Job 2: Type Check (runs in parallel with lint) ---
  type-check:
    name: Type Check
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.12"

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install mypy
          pip install -e ".[dev]"

      - name: Run mypy
        run: mypy src/

  # --- Job 3: Test (runs in parallel with lint and type-check) ---
  test:
    name: Test (Python ${{ matrix.python-version }})
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: ["3.11", "3.12", "3.13"]
      fail-fast: false

    steps:
      - uses: actions/checkout@v4

      - name: Set up Python ${{ matrix.python-version }}
        uses: actions/setup-python@v5
        with:
          python-version: ${{ matrix.python-version }}

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -e ".[dev]"

      - name: Run tests with coverage
        run: pytest --cov=src --cov-report=xml

      - name: Upload coverage
        if: matrix.python-version == '3.12'
        uses: codecov/codecov-action@v4
        with:
          file: coverage.xml
"""

print("GitHub Actions CI workflow:")
print(ci_workflow)

In [ ]:
# Branch Protection and Required Status Checks
#
# Branch protection rules ensure code quality by requiring certain
# conditions before merging to protected branches.

branch_protection = """
Branch protection for 'main':

  Settings > Branches > Branch protection rules > main

  [x] Require a pull request before merging
      [x] Require approvals: 1
      [x] Dismiss stale pull request approvals when new commits are pushed

  [x] Require status checks to pass before merging
      Required checks:
        - lint              (from CI workflow)
        - type-check        (from CI workflow)
        - test (3.11)       (from CI workflow matrix)
        - test (3.12)       (from CI workflow matrix)
        - test (3.13)       (from CI workflow matrix)

  [x] Require branches to be up to date before merging

  [x] Require conversation resolution before merging

  [ ] Include administrators (optional -- enforce rules on admins too)


Effect: A PR to main can only be merged when:
  1. At least 1 approval from a reviewer
  2. All CI checks pass (lint, type-check, all test matrix entries)
  3. Branch is up-to-date with main
  4. All review comments are resolved
"""

print(branch_protection)

## Project Structure: src-layout

The **src-layout** is the recommended project structure for Python packages. It prevents
accidental imports from the source directory during testing and ensures your installed
package is what gets tested.

In [ ]:
# src-layout project structure

project_structure = """
my-project/
|
|-- .github/
|   |-- workflows/
|       |-- ci.yml              # CI pipeline
|       |-- release.yml         # Release automation
|
|-- src/
|   |-- my_project/             # Package source code
|       |-- __init__.py         # Package init (version, public API)
|       |-- core.py             # Core business logic
|       |-- models.py           # Data models
|       |-- utils.py            # Utility functions
|       |-- py.typed            # PEP 561 marker for type info
|
|-- tests/
|   |-- conftest.py             # Shared test fixtures
|   |-- test_core.py            # Tests for core module
|   |-- test_models.py          # Tests for models module
|   |-- test_utils.py           # Tests for utils module
|
|-- docs/                       # Documentation (optional)
|   |-- index.md
|   |-- api.md
|
|-- .pre-commit-config.yaml     # Pre-commit hook configuration
|-- .gitignore                  # Git ignore rules
|-- .editorconfig               # Editor configuration
|-- pyproject.toml              # Project metadata, build config, tool config
|-- Makefile                    # Development task automation
|-- README.md                   # Project documentation
|-- LICENSE                     # License file
"""

print("Recommended project structure (src-layout):")
print(project_structure)

print("Why src-layout?")
print("  1. Prevents importing source directly during testing")
print("  2. Forces you to install the package (pip install -e .)")
print("  3. Tests run against the installed package, not loose files")
print("  4. Matches how end users will import your code")

In [ ]:
# pyproject.toml: The single source of truth
#
# pyproject.toml consolidates project metadata, build configuration,
# and tool settings into a single file.

pyproject_example = """
[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "my-project"
version = "1.2.0"
description = "A well-structured Python project"
readme = "README.md"
license = {text = "MIT"}
requires-python = ">=3.11"
authors = [
    {name = "Your Name", email = "you@example.com"},
]

# Runtime dependencies
dependencies = [
    "httpx>=0.27",
    "pydantic>=2.0",
]

# Optional dependency groups
[project.optional-dependencies]
dev = [
    "pytest>=8.0",
    "pytest-cov>=6.0",
    "mypy>=1.13",
    "ruff>=0.8",
    "pre-commit>=4.0",
]
docs = [
    "mkdocs>=1.6",
    "mkdocs-material>=9.5",
]

# Entry points (CLI commands)
[project.scripts]
my-tool = "my_project.cli:main"

# --- Tool configurations ---

[tool.ruff]
target-version = "py311"
line-length = 88

[tool.ruff.lint]
select = ["E", "W", "F", "I", "UP", "B", "SIM", "RUF"]

[tool.mypy]
python_version = "3.11"
strict = true
warn_return_any = true
warn_unused_configs = true

[tool.pytest.ini_options]
addopts = ["-ra", "--strict-markers", "--tb=short"]
testpaths = ["tests"]
"""

print("pyproject.toml (comprehensive example):")
print(pyproject_example)

## Makefile Patterns

A **Makefile** provides a simple, standard interface for common development tasks.
Developers can type `make test` or `make check` without memorizing tool-specific
commands. Makefiles use tabs for indentation (not spaces).

In [ ]:
# Makefile for a Python project

makefile_content = """
# Makefile
.PHONY: help install check lint format typecheck test clean

# Default target: show help
help:  ## Show this help message
\t@grep -E '^[a-zA-Z_-]+:.*?## .*$$' $(MAKEFILE_LIST) | \\
\t\tsort | \\
\t\tawk 'BEGIN {FS = ":.*?## "}; {printf "  \\033[36m%-15s\\033[0m %s\\n", $$1, $$2}'

install:  ## Install the project and dev dependencies
\tpython -m pip install --upgrade pip
\tpip install -e ".[dev]"
\tpre-commit install

lint:  ## Run linter (ruff check)
\truff check .

format:  ## Auto-format code (ruff format)
\truff format .
\truff check --fix .

typecheck:  ## Run type checker (mypy)
\tmypy src/

test:  ## Run tests with coverage
\tpytest --cov=src --cov-report=term-missing

check: lint typecheck test  ## Run all checks (lint + typecheck + test)

clean:  ## Remove build artifacts and caches
\trm -rf build/ dist/ .eggs/ *.egg-info/
\trm -rf .pytest_cache/ .mypy_cache/ .ruff_cache/
\trm -rf htmlcov/ .coverage coverage.xml
\tfind . -type d -name __pycache__ -exec rm -rf {} +
"""

print("Makefile for Python development:")
print(makefile_content)

print("Usage:")
print("  make help       # Show available targets")
print("  make install    # Set up the development environment")
print("  make check      # Run all quality checks")
print("  make format     # Auto-format the codebase")
print("  make test       # Run the test suite")
print("  make clean      # Clean up generated files")
print()
print("Key Makefile concepts:")
print("  .PHONY          # Declare targets that are not files")
print("  target: prereqs  # prereqs run before the target")
print("  check: lint typecheck test  # 'check' depends on all three")

## Dependency Management

Managing dependencies properly ensures reproducible builds, prevents supply chain
attacks, and avoids version conflicts. Modern Python offers several approaches
to dependency locking and security scanning.

In [ ]:
# Dependency management strategies

dep_management = """
Dependency Management Strategies:

1. ABSTRACT DEPENDENCIES (pyproject.toml)
   - Specify minimum version constraints
   - Used for libraries (consumed by other packages)
   
   dependencies = [
       "httpx>=0.27",         # Minimum version
       "pydantic>=2.0,<3",   # Version range
   ]

2. LOCK FILES (pip-tools, uv, poetry)
   - Pin exact versions for reproducible installs
   - Used for applications (deployed directly)
   
   # Using pip-tools:
   pip-compile pyproject.toml -o requirements.lock
   pip install -r requirements.lock
   
   # Using uv (faster alternative):
   uv lock
   uv sync

3. LOCK FILE EXAMPLE (requirements.lock)
   # Generated by pip-compile -- DO NOT EDIT
   certifi==2024.12.14
   h11==0.14.0
   httpcore==1.0.7
   httpx==0.28.1
   idna==3.10
   pydantic==2.10.4
   pydantic-core==2.27.2
"""

print(dep_management)

security = """
Dependency Security Scanning:

  # pip-audit: check for known vulnerabilities
  pip install pip-audit
  pip-audit
  
  # GitHub Dependabot (automatic):
  # .github/dependabot.yml
  version: 2
  updates:
    - package-ecosystem: pip
      directory: "/"
      schedule:
        interval: weekly
      open-pull-requests-limit: 10
  
  # Safety check in CI:
  - name: Security audit
    run: pip-audit --strict
"""

print(security)

## Release Workflow: Tagging and Semantic Versioning

**Semantic Versioning** (SemVer) uses a `MAJOR.MINOR.PATCH` scheme:
- **MAJOR**: Breaking changes (incompatible API changes)
- **MINOR**: New features (backward-compatible)
- **PATCH**: Bug fixes (backward-compatible)

A release workflow automates tagging, changelog generation, and package publishing.

In [ ]:
# Semantic versioning examples

semver_examples = """
Semantic Versioning (SemVer): MAJOR.MINOR.PATCH

  1.0.0  Initial stable release
  1.0.1  Bug fix (patch)
  1.1.0  New feature added (minor)
  1.1.1  Bug fix in new feature (patch)
  2.0.0  Breaking API change (major)
  2.0.0-rc.1  Release candidate (pre-release)

When to bump each number:
  PATCH (1.0.x): Bug fixes, documentation updates
  MINOR (1.x.0): New features, deprecations
  MAJOR (x.0.0): Removed features, changed APIs, breaking changes

Pre-release versions:
  1.0.0-alpha.1   Early development
  1.0.0-beta.1    Feature complete, testing
  1.0.0-rc.1      Release candidate
"""

print(semver_examples)

# Working with git tags for releases
print("Git tagging for releases:")
print("  git tag v1.2.0                       # Create a lightweight tag")
print("  git tag -a v1.2.0 -m 'Release 1.2.0' # Create an annotated tag")
print("  git push origin v1.2.0               # Push tag to remote")
print("  git push origin --tags               # Push all tags")
print("  git tag -l 'v1.*'                    # List matching tags")

In [ ]:
# GitHub Actions release workflow

release_workflow = """
# .github/workflows/release.yml
name: Release

on:
  push:
    tags:
      - 'v*'  # Trigger on version tags (v1.0.0, v2.1.3, etc.)

permissions:
  contents: write  # Needed to create GitHub releases
  id-token: write  # Needed for trusted publishing to PyPI

jobs:
  # First, run all CI checks
  ci:
    uses: ./.github/workflows/ci.yml  # Reuse the CI workflow

  # Then build and publish
  release:
    name: Build and Publish
    needs: ci  # Only run after CI passes
    runs-on: ubuntu-latest

    steps:
      - uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.12"

      - name: Install build tools
        run: pip install build

      - name: Build package
        run: python -m build

      - name: Create GitHub Release
        uses: softprops/action-gh-release@v2
        with:
          files: dist/*
          generate_release_notes: true

      - name: Publish to PyPI
        uses: pypa/gh-action-pypi-publish@release/v1
        # Uses trusted publishing (no API token needed)
"""

print("GitHub Actions release workflow:")
print(release_workflow)

In [ ]:
# Changelog management: keeping users informed

changelog_example = """
# CHANGELOG.md
#
# Format based on Keep a Changelog (https://keepachangelog.com)

## [Unreleased]
### Added
- Support for Python 3.13

## [1.2.0] - 2025-12-15
### Added
- New `export_csv()` method on Report class
- CLI flag `--output-format` for choosing export format

### Changed
- Improved error messages for invalid configuration files

### Fixed
- Race condition in concurrent file processing (#142)

## [1.1.0] - 2025-09-01
### Added
- Initial support for async operations
- Configuration file validation

### Deprecated
- `process_sync()` -- use `process()` with `async=False` instead

## [1.0.0] - 2025-06-01
### Added
- Initial stable release
- Core processing pipeline
- CLI interface
- Full documentation
"""

print("Changelog (Keep a Changelog format):")
print(changelog_example)

print("Release process summary:")
print("  1. Update version in pyproject.toml")
print("  2. Update CHANGELOG.md (move Unreleased -> new version)")
print("  3. Commit: git commit -m 'Release v1.2.0'")
print("  4. Tag:    git tag -a v1.2.0 -m 'Release 1.2.0'")
print("  5. Push:   git push origin main --tags")
print("  6. CI runs, then release workflow builds and publishes")

In [ ]:
# Putting it all together: the development workflow

full_workflow = """
Complete Development Workflow:

  SETUP (one time)
  ================
  $ git clone https://github.com/user/my-project.git
  $ cd my-project
  $ python -m venv .venv
  $ source .venv/bin/activate
  $ make install           # pip install -e ".[dev]" + pre-commit install

  DAILY DEVELOPMENT
  =================
  $ git checkout -b feature/my-feature    # Create feature branch
  
  # ... write code and tests ...
  
  $ make check             # Run lint + typecheck + tests locally
  $ git add -p             # Stage changes interactively
  $ git commit             # Pre-commit hooks run automatically
  $ git push -u origin feature/my-feature
  
  # Open a Pull Request on GitHub
  # CI runs lint, type-check, and tests in parallel
  # Reviewer approves
  # Merge to main

  RELEASE
  =======
  $ vim pyproject.toml     # Bump version
  $ vim CHANGELOG.md       # Update changelog
  $ git commit -m 'Release v1.2.0'
  $ git tag -a v1.2.0 -m 'Release 1.2.0'
  $ git push origin main --tags
  # Release workflow builds, tests, and publishes automatically
"""

print(full_workflow)

## Summary

### Key Takeaways

| Area | Tool / Practice | Purpose |
|------|----------------|----------|
| **Pre-commit** | `.pre-commit-config.yaml` | Catch issues before they enter the repo |
| **CI/CD** | GitHub Actions (`.github/workflows/`) | Automated lint, type-check, test |
| **Parallel jobs** | `lint`, `type-check`, `test` | Fast feedback on PRs |
| **Matrix testing** | `strategy.matrix` | Test across Python versions |
| **Branch protection** | Required status checks | Enforce quality gates |
| **src-layout** | `src/my_project/` | Correct package structure |
| **pyproject.toml** | Single config file | Metadata + all tool configs |
| **Makefile** | `make check`, `make test` | Standard developer interface |
| **Lock files** | `pip-compile`, `uv lock` | Reproducible installs |
| **Security** | `pip-audit`, Dependabot | Vulnerability scanning |
| **SemVer** | `MAJOR.MINOR.PATCH` | Communicate change impact |
| **Release** | Git tags + GitHub Actions | Automated build and publish |

### Best Practices
- Install pre-commit hooks as the first step when joining a project
- Run `make check` locally before pushing to catch issues early
- Keep CI jobs parallel to minimize feedback time on pull requests
- Use the src-layout to prevent import confusion during testing
- Consolidate all configuration in `pyproject.toml` where possible
- Pin dependencies with lock files for applications, use ranges for libraries
- Follow semantic versioning to communicate the impact of changes
- Automate releases with CI/CD to eliminate manual packaging errors